# Лекция 5.2 "Монады. Часть 2"

__План:__
- Writer Monad
  - Logging evaluation
  - Result
- Reader Monad
  - Что? Зачем?
  - Result
- Type helpers
  - Typed holes
  - -XPartialTypeSignatures
- State Monad
  - Зачем?
  - Примеры
- Cont Monad
  - Continuation Passing Style (CPS)
  - Cont Monad

## Writer Monad

### Logging evaluation

Так как _`Haskell`_ это функциональный ЯП, то сделать логирование для операций нетривиально. Поэтому используют обертки над типами, чтобы в поля `String` записывать лог. 

Пример:

```
type IntWithLog = (Int, String)

binPow :: Int -> Int -> IntWithLog
binPow 0 _      = (1, "")
binPow n a
    | even n    = let (b, msg) = binPow (n `div` 2) a 
                  in (b * b, msg ++ "Square " ++ show b ++ "\n")
    | otherwise = let (b, msg) = binPow (n - 1) a
                  in (a * b, msg ++ "Mul " ++ show a ++ " and " ++ show b ++ "\n")

ghci> putStr $ snd $ binPow 3 2
Mul 2 and 1
Square 2
Mul 2 and 4
```

Согласитесь, не особо удобно. Поэтому давайте быстренько намутим какую-то более удобную структуру. Например, монаду:

```
newtype Writer w a = Writer { runWriter :: (a, w) } -- a is value, w is log
instance Monoid w => Monad (Writer w) where
    return :: a -> Writer w a
    return a = Writer (a, mempty)
    
    (>>=) :: Writer w a -> (a -> Writer w b) -> Writer w b
    Writer (a, oldLog) >>= f = let Writer (b, newLog) = f a 
                               in Writer (b, oldLog <> newLog)

-- Вспомогательные функции --
tell       :: w -> Writer w ()
execWriter :: Writer w a -> w
writer     :: (a, w) -> Writer w a
```

Теперь наше логирование будет выглядеть куда проще:

```
binPow :: Int -> Int -> Writer String Int
binPow 0 _      = return 1
binPow n a
    | even n    = binPow (n `div` 2) a >>= \b -> 
                  writer (b * b, "Square " ++ show b ++ "\n")
    | otherwise = binPow (n - 1) a >>= \b -> 
                  writer (a * b, "Mul " ++ show a ++ " and " ++ show b ++ "\n")
```

__Но__ в реальности такого типа не существует.  
Потому что __Writer__ это особый случай более сложного типа — __`WriterT`__, что, в свою очередь, значит:
1. There's no __`Writer`__ constructor. You can't pattern match on it.
2. There's no __`Writer`__ constructor. Use __`writer`__ function to create.

Более того, в реальности существует две библиотеки, где можно найти __`Writer`__: __`transformers`__ & __`mtl`__:

```
-- transformers -- (only types and functions)
import Control.Monad.Trans.Writer (Writer, writer, ... )

-- mtl -- (transformers + extra interfaces + shorter imports)
import Control.Monad.Writer (Writer, writer, ... )
```

### Вывод

Чистый __`Writer`__ нам не пригодится :((  

Есть парочка кейсов, где никак не обойтись без него:
  - Pure logging (хотим обрабатывать логи и писать для них тесты)
  - Collecting intermediate results (сбор промежуточных результатов)

## Reader Monad

### Что? Зачем?

Предположим, нам надо получить доступ к __глобальной переменной__. В ФП такого сделать __нельзя__, поэтому первое, что приходит на ум - это явно передавать значение в виде аргумента функции.  
Идея, конечно хорошая, но неприятно. А что если сделать обертку над нашей глобальной переменной. Что-то тпио недоконца определенного типа (:kind * -> * -> *). Неплохое решение, тогда мы сможем спокойно иметь под капотом желанное значение. Реализуется такая вот идея с помощью __Reader monad__:

```
newtype Reader e a = Reader { runReader :: e -> a }

instance Monad (Reader e) where
    return :: a -> Reader e a
    return a = Reader $ \_ -> a

    (>>=) :: Reader e a -> (a -> Reader e b) -> Reader e b
    m >>= f = Reader $ \r -> runReader (f $ runReader m r) r

-- Вспомогательные функции --
ask   :: Reader e e                            -- get whole env
asks  :: (e -> a) -> Reader e a                -- get part of env
local :: (e -> b) -> Reader e a -> Reader b a  -- change env locally
```

Таким образом, __`Reader`__ monad нужна, чтобы скрыто проталкивать неизменяемую __`env`__ (environment) во все вычисления, чего мы и добивались.

### Вывод

Пример работы:

```
inEnv :: Int -> Reader Environment Bool
inEnv i = asks (elem i . ids)

anyInEnv :: (Int, Int) -> Reader Environment Bool
anyInEnv (i, j) = inEnv i ||^ inEnv j

checkNeighbours :: Int -> Reader Environment (Maybe String)
checkNeighbours i = 
  asks (`near` i) >>= \pair ->
  anyInEnv pair   >>= \res  ->
  if res 
  then Just <$> asks (`name` i)
  else pure Nothing

ghci> runReader (checkNeighbours 0) $ Environment [1] show (const (1,3))
Just "0"
ghci> runReader (checkNeighbours 0) $ Environment [2] show (const (1,3))
Nothing
```

Итак, зачем же нам нужен __`Reader`__?

1. __`Reader`__ is __the most important__ monad in real life.
2. You don't need to pass configs and parameters explicitly. (не надо постоянно передавать __`env`__)
3. You can't accidentally change environment because you don't have direct access to it. (__`env`__ нельзя изменить, даже случайно)
4. Your implementations can be polymorphic and can work with different parts of config. (можем работать только с частью конфига)

## Type helpers

### Typed holes

Что такое __typed holes__?

_ to the left of = means I don't care about this pattern  
_ to the right of = means __typed hole__

```
foo :: a -> b
foo x = _
```

Выведет ошибку, с подсказкой, что подставить на место "дырки".

__Named typed holes__:

```
mfold :: [Maybe Bool] -> [Either Bool ()]
mfold = foldr _f _z
```

Выведет ошибку с подсказкой для каждой "дырки" (_f | _z).

Приминение "дырок":

```
join :: Monad m => m (m a) -> m a
join m = _

    • Found hole: _ :: m a

-- >>
join :: Monad m => m (m a) -> m a
join m = _k >>= _f

    • Found hole: _k :: m a0
    • Found hole: _f :: a0 -> m a

-- >>
join :: Monad m => m (m a) -> m a
join m = m >>= _f

    • Found hole: _f :: m a -> m a

-- >> Ура!!! Мы справились. 
join :: Monad m => m (m a) -> m a
join m = m >>= id
```

### -XPartialTypeSignatures

При подключении засширешия __`-XPartialTypeSignatures`__ мы можем прибегать к помощи компилятора не только внутри реализации функций, но и в их описании:

```
{-# LANGUAGE PartialTypeSignatures #-}

-- Пример 1 --

foo :: _ -> Bool
foo x = not x

Hole.hs:3:8: warning: [-Wpartial-type-signatures]
    • Found type wildcard ‘_’ standing for ‘Bool’
    • In the type signature: foo :: _ -> Bool
  |
3 | foo :: _ -> Bool
  |        ^

-- Пример 2 --

arbitCs :: _ => a -> String
arbitCs x = show (succ x) ++ show (x == x)

Hole.hs:6:12: warning: [-Wpartial-type-signatures]
    • Found type wildcard ‘_’ standing for ‘(Eq a, Enum a, Show a)’
      Where: ‘a’ is a rigid type variable bound by
               the inferred type of
               arbitCs :: (Eq a, Enum a, Show a) => a -> String
               at Hole.hs:7:1-42
    • In the type signature: arbitCs :: _ => a -> String
  |
6 | arbitCs :: _ => a -> String
  |            ^
```

## State Monad

### Зачем?

Evaluation modifies state:
- __Imperative style:__ change value in variable  
- __Functional style:__ create new variable with new value

И чтобы не создавать каждый раз новых переменных, давайте сделаем структуру, которая умеет возвращать пару : (результат, новое состояние)

```
newtype State s a = State { runState :: s -> (a, s) }

instance Monad (State s) where
    return a       = State $ \s -> (a, s)
    oldState >>= f = State $ \s -> let (a, newState) = runState oldState s
                                   in runState (f a) newState

-- Полезные функции --
get       :: State s s
put       :: s -> State s ()
modify    :: (s -> s) -> State s ()
gets      :: (s -> a) -> State s a
withState :: (s -> s) -> State s a -> State s a
evalState :: State s a -> s -> a
execState :: State s a -> s -> s
```

Теперь мы внутри bind-а создаем новые переменные и запускаем вычисления на них.

### Примеры

Примеры:

```
type Stack = [Int]

pop :: State Stack Int
pop = state $ \(x:xs) -> (x, xs)

push :: Int -> State Stack ()
push x = state $ \xs -> ((), x:xs)

stackOps :: State Stack Int
stackOps = pop >>= \x -> push 5 >> push 10 >> return x

gchi> evalState stackOps [1, 2, 3]
1
ghci> execState stackOps [1, 2, 3]
[10,5,2,3]

-- Repeating monadic action --
pop :: State Stack Int
pop = state $ \(x:xs) -> (x, xs)

multipop :: Int -> State Stack [Int]
multipop n = replicateM n pop

ghci> :t replicateM
replicateM :: Monad m => Int -> m a -> m [a]

ghci> runState (multipop 3) [1..10]
([1,2,3], [4,5,6,7,8,9,10])

-- Stack Operations --
data StackOperation = Pop | Push Int

doOperations :: [StackOperation] -> State Stack ()
doOperations ops = forM_ ops $ \case  -- -XLambdaCase extension
    Pop    -> pop >> return ()
    Push n -> push n

ghci> :t forM_
forM_ :: (Monad m, Foldable t) => t a -> (a -> m b) -> m ()

ghci> execState (doOperations [Pop, Pop, Push 100]) [1..5]
[100,3,4,5]
```

## Cont Monad

### Continuation Passing Style (CPS)

```
add :: Int -> Int -> Int
add x y = x + y

addCPS :: Int -> Int -> (Int -> r) -> r
addCPS x y onDone = onDone (x + y)
```

CPS можно предствить так:  
Что-то посчитанное обернул в функцию и вернул. 

Ещё примеры:

```
-- Normal evaluations --
square :: Int -> Int
square x = x * x

pythagoras :: Int -> Int -> Int
pythagoras x y = (+) (square x) (square y)

-- CPS --
addCPS :: Int -> Int -> ((Int -> r) -> r)
addCPS x y = \k -> k (x + y)

squareCPS :: Int -> ((Int -> r) -> r)
squareCPS x = \k -> k (square x)

pythagorasCPS :: Int -> Int -> ((Int -> r) -> r)
pythagorasCPS x y = \k ->  -- k :: Int -> r
    squareCPS x  $ \x2 ->
    squareCPS y  $ \y2 ->
    addCPS x2 y2 $ k       -- addCPS x2 y2 :: (Int -> r) -> r
```

Чтобы просто вывести результат, можно передать в виде callback() функции функцию __id__

```
ghci> pythagorasCPS 3 4 id
25
```

Вспомним про оператор $:

```
ghci> :t ($)
($) :: (a -> b) -> a -> b

gchi> :t ($ 2)
($ 2) :: Num a => (a -> b) -> b

ghci> map ($ 2) [(3*), (2+),(1-)]
[6,4,-1]
```

Таким образом можно трактовать "$ 2" как "возьми 2 и примени к нему переданную функцию".


### Cont Monad

Внезапно существуют __монады__ для CPS вычислений:

```
newtype Cont r a = Cont { runCont :: (a -> r) -> r }

instance Monad (Cont r) where
    return :: a -> Cont r a
    return a = Cont ($ a)

    (>>=) :: Cont r a -> (a -> Cont r b) -> Cont r b
    Cont arr >>= f = Cont $ \br -> arr $ \a -> runCont (f a) br

    -- arr :: (a -> r) -> r
    -- br  :: (b -> r)
    -- f   :: a -> Cont r b
```

Пример работы с __`Cont Monad`__:

```
addCPS :: Int -> Int -> Cont r Int
addCPS x y = return $ x + y

squareCPS :: Int -> Cont r Int
squareCPS = return . square

pythagorasCPS :: Int -> Int -> Cont r Int
pythagorasCPS x y = squareCPS x >>= \x2 ->
                    squareCPS y >>= \y2 ->
                    addCPS x2 y2
```